In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import pandas as pd

In [13]:
dataChurn = pd.read_parquet('DataChurn.parquet')[['Comentários','ID','Nome','Localização','Tipo de Serviço']]

In [15]:
loader = DataFrameLoader(dataChurn,page_content_column='Comentários')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [41]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever(),return_source_documents=True)

In [42]:
query = ''
while True:
    query = input()
    if query == 'Sair':
        break
    print(qa_chain({"query": query}))

{'query': 'Me traga um comentário de alguem feliz com o serviço', 'result': 'Estou muito satisfeito com o serviço de telefonia móvel. A cobertura é excelente e a internet é rápida. Além disso, não tive nenhum problema com a conexão até o momento. As ligações são sempre claras e o atendimento ao cliente é muito bom. Estou realmente feliz com esse serviço.', 'source_documents': [Document(page_content='Estou satisfeita com o serviço de telefonia móvel A cobertura é boa e a internet é rápida', metadata={'ID': '1011', 'Localização': 'São Paulo/SP/São Paulo', 'Nome': 'Isabela Fernandes', 'Tipo de Serviço': 'Telefonia Móvel'}), Document(page_content='Estou satisfeita com o serviço de telefonia móvel A cobertura é boa e a internet é rápida', metadata={'ID': '1011', 'Localização': 'São Paulo/SP/São Paulo', 'Nome': 'Isabela Fernandes', 'Tipo de Serviço': 'Telefonia Móvel'}), Document(page_content='Estou satisfeita com o serviço de telefonia móvel A cobertura é boa e a internet é rápida', metadat